In [ ]:
!git clone https://gitfront.io/r/mattbit/NznKzjgfZvMP/mlr-technical-test.git
%cd mlr-technical-test


Cloning into 'mlr-technical-test'...
Fetching objects: 21, done.
/content/mlr-technical-test


In [ ]:
!pip install pdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 29.9 MB/s eta 0:00:00


In [ ]:
!ls /content/mlr-technical-test/


chameleon.md  paper-review.md  pdm.lock  pyproject.toml  README.md  src  tests


In [ ]:
!pdm init


pyproject.toml already exists, update it now.
Please enter the Python interpreter to use
 0. cpython@3.11 (/usr/local/bin/python)
 1. cpython@3.11 (/usr/bin/python3.11)
 2. cpython@3.10 (/usr/bin/python3.10)
 3. cpython@3.11 (/usr/bin/python3)
Please select (0): 0
Virtualenv is created successfully at /content/mlr-technical-test/.venv
Project name (mlr-technical-test): mlr-technical-test
Project version (0.1.0): 0.1.0
Do you want to build this project for distribution(such as wheel)?
If yes, it will be installed by default when running `pdm install`. [y/n] (n): n
License(SPDX name) (MIT): MIT
Author name (): 
Author email (): 
Python requires('*' to allow any) (==3.11.*): ==3.11.*
Project is initialized successfully


In [ ]:
!pdm install


Updating the lock file...
DEPRECATED: `cross_platform` strategy is deprecated in favor of the new lock targets.
See docs: http://pdm-project.org/en/latest/usage/lock-targets/
⠸ 0:00:00 Resolve for environment (==3.11.*) None
⠦ 0:00:00 Resolve for environment (==3.11.*) None
⠋ 0:00:00 Resolve for environment (==3.11.*) None
⠸ 0:00:01 Resolve for environment (==3.11.*) None
⠧ 0:00:01 Resolve for environment (==3.11.*) None
⠋ 0:00:01 Resolve for environment (==3.11.*) None
⠸ 0:00:01 Resolve for environment (==3.11.*) None
⠦ 0:00:02 Resolve for environment (==3.11.*) 2 resolved, 10 to resolve
⠋ 0:00:02 Resolve for environment (==3.11.*) 2 resolved, 10 to resolve
⠸ 0:00:02 Resolve for environment (==3.11.*) 2 resolved, 10 to resolve
⠦ 0:00:02 Resolve for environment (==3.11.*) 2 resolved, 10 to resolve
⠏ 0:00:03 Resolve for environment (==3.11.*) 2 resolved, 10 to resolve
⠸ 0:00:03 Resolve for environment (==3.11.*) 2 resolved, 10 to resolve
⠦ 0:00:03 Resolve for environment (==3.11.*) 2 re

In [7]:
!pdm venv activate


  deprecated-cross-platform: Lock strategy `cross_platform` has been deprecated in favor of lock 
targets.
Run pdm fix to fix all or pdm fix <name> to fix individual problem.
source /content/mlr-technical-test/.venv/bin/activate


In [9]:
!pdm fix


Fixing deprecated-cross-platform... ✔


In [10]:
!python chameleon.md

  File "/content/mlr-technical-test/chameleon.md", line 3
    Chamaleon is a library of probes that can analyze a text classification model
                   ^^^^^^^
SyntaxError: invalid syntax


In [13]:
!ls -R src/chameleon



src/chameleon:
__init__.py  models  probes

src/chameleon/models:
base.py  huggingface.py  __init__.py

src/chameleon/probes:
base.py  __init__.py


In [14]:
%%writefile src/chameleon/models/huggingface.py
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

class HuggingFaceModel:
    """
    A wrapper class for using a Hugging Face sentiment analysis model.
    """

    def __init__(self, model_name="lxyuan/distilbert-base-multilingual-cased-sentiments-student"):
        """
        Initializes the tokenizer and model from the Hugging Face library.
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)

    def predict(self, sentence):
        """
        Analyzes the sentiment of the given sentence and returns sentiment scores.
        """
        inputs = self.tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        scores = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
        return {
            "positive": scores[0],
            "neutral": scores[1],
            "negative": scores[2]
        }


Overwriting src/chameleon/models/huggingface.py


In [15]:
%%writefile src/chameleon/models/__init__.py
from chameleon.models.huggingface import HuggingFaceModel


Overwriting src/chameleon/models/__init__.py


In [16]:
%%writefile src/chameleon/probes/sentiment_probe.py
from nltk.metrics.distance import edit_distance
from chameleon.models import HuggingFaceModel

class MySuperEfficientProbe:
    """
    A probe that finds a sentence with similar sentiment scores
    to a target sentence while enforcing constraints on Levenshtein distance and length.
    """

    def __init__(self, model, target_sentence, decimal_places=5):
        self.model = model
        self.target_sentence = target_sentence
        self.decimal_places = decimal_places

    def get_sentiment_scores(self, sentence):
        return self.model.predict(sentence)

    def find_matching_sentence(self, candidate_sentences, epsilon=1e-3):
        """
        Finds a sentence that has sentiment scores matching the target sentence
        when rounded to the specified decimal places.
        """
        target_scores = self.get_sentiment_scores(self.target_sentence)
        rounded_target_scores = {k: round(v, self.decimal_places) for k, v in target_scores.items()}

        for candidate_sentence in candidate_sentences:
            if not (40 <= len(candidate_sentence) <= 60):
                continue
            if edit_distance(self.target_sentence, candidate_sentence) < 30:
                continue

            candidate_scores = self.get_sentiment_scores(candidate_sentence)
            rounded_candidate_scores = {k: round(v, self.decimal_places) for k, v in candidate_scores.items()}

            if all(abs(rounded_candidate_scores[k] - rounded_target_scores[k]) <= epsilon for k in rounded_target_scores):
                return candidate_sentence, candidate_scores

        return None, target_scores


Writing src/chameleon/probes/sentiment_probe.py


In [17]:
%%writefile src/chameleon/probes/__init__.py
from chameleon.probes.sentiment_probe import MySuperEfficientProbe


Overwriting src/chameleon/probes/__init__.py


In [18]:
!ls src/chameleon/models
!ls src/chameleon/probes


base.py  huggingface.py  __init__.py
base.py  __init__.py  sentiment_probe.py


In [19]:
!pdm run test


======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-7.4.3, pluggy-1.3.0
rootdir: /content/mlr-technical-test
collected 0 items / 1 error                                                                        

============================================== ERRORS ==============================================
_________________________ ERROR collecting tests/test_huggingface_model.py _________________________
ImportError while importing test module '/content/mlr-technical-test/tests/test_huggingface_model.py'.
Hint: make sure your test modules/packages have valid Python names.
Traceback:
/usr/lib/python3.11/importlib/__init__.py:126: in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
tests/test_huggingface_model.py:2: in <module>
    from chameleon.models import HuggingFaceModel, BaseModel
E   ModuleNotFoundError: No module named 'chameleon'
===========================

In [30]:
%%writefile test_huggingface_model.py
import sys
sys.path.append("/content/mlr-technical-test/src")

from chameleon.models import HuggingFaceModel

# Initialize the model
model = HuggingFaceModel()

# Define a test sentence
test_sentence = "I love sunny days"

# Get sentiment scores
scores = model.predict(test_sentence)

print("Test Sentence:", test_sentence)
print("Sentiment Scores:", scores)


Overwriting test_huggingface_model.py


In [31]:
!python test_huggingface_model.py


2025-02-23 12:12:41.723347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740312761.736299    7235 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740312761.739844    7235 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 12:12:41.756799: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Test Sentence: I love sunny days
Sentiment Scores: {'positive': 0.9439314, 'neutral': 0.038848154, 'negative': 0.017220419}
